------------------------------------------------- Nuovo script -------------------------------------------------

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, lit, collect_list, col, regexp_replace, size, expr, count, array_contains

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
    .master("local") \
    .appName("MyFirstSparkApplication") \
    .getOrCreate()
# master contains the URL of your remote spark instance or 'local'


In [3]:
vectASIA = ["Department of Statistics, Rajshahi University, Rajshahi, Bangladesh",
            "Department of Computer Science & Engineering, BUET, Dhaka, Bangladesh",
            'Department of System Design and Informatics, Kyushu Institute of Technology, Fukuka, Japan',
            "Corporate Research and Development Center, Toshiba Corporation, Kawasaki, Japan",
            "Institute of Industrial Science, University of Tokyo, Meguro-Ku, Japan", "continent",
            "Department of Human and Artificial Intelligence System, Graduate School of Engineering, University of Fukui, Japan",
            "Department of Computer Science, Faculty of Engineering, Yamanashi University, Kofu, Japan",
            "Nippon Hoso Kyokai", "Shinshu University", 'Nagano Prefectural College']

vectEU = ["Università della Calabria", "Università degli Studi di Modena e Reggio Emilia", "Politecnico di Milano",
          'Università degli Studi di Firenze'
          "University of Milano-Bicocca",
          "Cognitive Interaction Technology, Center of Excellence and Applied Informatics, Faculty of Technology, Bielefeld University, Bielefeld, Germany#TAB#",
          "Department of Information Systems and Business Administration, Johannes Gutenberg-Universität Mainz, Jakob-Welder-Weg 9, Mainz 55128, Germany",
          "Dept. of Computer Science, Uni. Politècnica de Catalunya, Spain#TAB#", "university of malaga",
          "Computer Laboratory, University Of Cambridge", "New England Aquarium",
          "Unconventional Computing Group, University of the West of England, Bristol, UK BS16 1QY#TAB#",
          "ESIEE-Paris", "UNIVERSITÉ LAVAL", "University of Southern Denmark", "University of Lisbon",
          "#N#University of Glasgow#N#"]

vectAMERICA = ['The University of Chicago Press, Chicago, USA']

author_df = spark.read.json(
    "authors.json")
fos_df = spark.read.json(
    "fos.json")
publication_df = spark.read.json(
    "publications.json")
venue_df = spark.read.json(
    "venues.json")
rel_df = spark.read.json(
    "rel_dw.json")



In [5]:
# Query 0
fos_df.filter(col("name") == "Artificial intelligence") \
    .join(rel_df, fos_df.id == rel_df.fos_id, "inner") \
    .join(publication_df, rel_df.pub_id == publication_df.id, "inner") \
    .select("title").show(truncate=False)


+--------------------------------------------------------------------------------------------------+
|title                                                                                             |
+--------------------------------------------------------------------------------------------------+
|A methodology for the physically accurate visualisation of roman polychrome statuary              |
|Comparison of GARCH, Neural Network and Support Vector Machine in Financial Time Series Prediction|
|COMPARING GNG3D AND QUADRIC ERROR METRICS METHODS TO SIMPLIFY 3D MESHES                           |
|Vectorial fast correlation attacks.                                                               |
|Improved Secret Image Sharing Method By Encoding Shared Values With Authentication Bits           |
|Identifying Psychological Theme Words from Emotion Annotated Interviews                           |
|A COMPUTATIONAL SALIENCY MODEL INTEGRATING SACCADE PROGRAMMING                            

In [6]:
# QUERY 1
author_df.withColumnRenamed("id", "authorId") \
  .filter(col("affiliation").like("%Politecnico%")) \
  .limit(5) \
  .join( publication_df, expr("array_contains(authors, authorId)") ) \
  .select(col("title").alias("publicationTitle"), col("name").alias("authorName"), "affiliation") \
  .show(truncate=False)


+--------------------------------------------------------------+-----------------+---------------------+
|publicationTitle                                              |authorName       |affiliation          |
+--------------------------------------------------------------+-----------------+---------------------+
|Vectorial fast correlation attacks.                           |Jovan Dj. Golic  |Politecnico di Milano|
|Vectorial fast correlation attacks.                           |Guglielmo Morgari|Politecnico di Bari  |
|A COMPUTATIONAL SALIENCY MODEL INTEGRATING SACCADE PROGRAMMING|Tien Ho-Phuoc    |Politecnico di Milano|
|Cleaneval: a Competition for Cleaning Web Pages               |Serge Sharoff    |Politecnico di Bari  |
|On the Design of Local Network Interfaces.                    |David J. Farber  |Politecnico di Milano|
+--------------------------------------------------------------+-----------------+---------------------+



WHERE, IN Nested Query. What can we do? In SQL something like:

SELECT paper
FROM papers
WHERE x in fos IN 
  (SELECT fos
  FROM fos's
  WHERE fos.name like %computer%
  )

SELECT paper
FROM papers
WHERE x in references IN 
  (SELECT paper
  FROM publications
  WHERE publication.authors.length = 1
  )

In [ ]:
# QUERY 2 - WHERE, IN Nested Query
publication_df \
  .select(col("title"), col("authors"), col("references")) \
  .filter(array_contains(col("references"),
    publication_df.select(col("id"), col("authors"))\
      .filter(size("authors") == 1)\
      .select(col("id"))
  ))


In [23]:
computerScience_fos = fos_df.filter(col("name").contains("omputer")).select(col("id")).collect()
computerScience_fos = [csf[0] for csf in computerScience_fos]

computerScience_pub = rel_df.filter(col("fos_id").isin(computerScience_fos)).select(col("pub_id")).distinct().collect()
computerScience_pub = [csp[0] for csp in computerScience_pub]

publication_df.filter(col("id").isin(computerScience_pub)).select(col("id"), col("title")).show()

+-----+--------------------+
|   id|               title|
+-----+--------------------+
| 1091|Preliminary Desig...|
| 1674|A methodology for...|
| 1688|Comparison of GAR...|
| 5411|COMPARING GNG3D A...|
| 5781|Vectorial fast co...|
| 6522|Improved Secret I...|
| 6762|A Self-Stabilizin...|
| 8373|Formal agent-orie...|
| 8763|Fur Visualisation...|
|11796|The Role of the B...|
|11895|Speech training s...|
|12993|Software Evolutio...|
|13070|Knowledge Enginee...|
|13205|Design of an audi...|
|14870|A COMPUTATIONAL S...|
|15548|Cleaneval: a Comp...|
|15883|Leveraging legacy...|
|15901|A pedestrian navi...|
|16022|Algorithms for th...|
|16160|Logical Derivatio...|
+-----+--------------------+
only showing top 20 rows



In [7]:
computerScience_fos = fos_df.filter(col("name").contains("Computer")).select(col("id")).collect()
computerScience_fos = [csf[0] for csf in computerScience_fos]

count_cs_publications = rel_df.filter(col("fos_id").isin(computerScience_fos)).select(col("pub_id")).distinct().count()
print("percentage of publications that have fos with Computer:" + str(count_cs_publications/2500))


percentage of publications that have fos with Computer:0.804
